# FLDetector for FEMNIST

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

/scratch/gypsum-gpu083/4287636/ipykernel_72878/912229180.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import random
import copy
import time
from functools import reduce
from torchsummary import summary

import os
import sys
import pickle
sys.path.insert(0,'./utils/')
from logger import *
from eval import *
from misc import *

from sklearn.metrics import roc_auc_score
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from collections import defaultdict

from SGD import *
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [3]:
dataset = 'femnist'
bias = 0.1
net = 'cnn'
batch_size = 32
lr = 0.01
num_workers = 300
nepochs = 100
gpu = 3
seed = 41
nbyz = 84
byz_type = 'full_trim'
aggregation = 'trim'

In [4]:
import json
user_tr_data = []
user_tr_labels = []

for i in range(34):
    f = '/work/vshejwalkar_umass_edu/momin/leaf/data/femnist/data/train/all_data_%d_niid_0_keep_0_train_9.json'%i
    with open(f, 'r') as myfile:
        data=myfile.read()
    obj = json.loads(data)
    
    for user in obj['users']:
        user_tr_data.append(obj['user_data'][user]['x'])
        user_tr_labels.append(obj['user_data'][user]['y'])

user_te_data = []
user_te_labels = []

for i in range(34):
    f = '/work/vshejwalkar_umass_edu/momin/leaf/data/femnist/data/test/all_data_%d_niid_0_keep_0_test_9.json'%i
    with open(f, 'r') as myfile:
        data=myfile.read()
    obj = json.loads(data)
    
    for user in obj['users']:
        user_te_data.append(obj['user_data'][user]['x'])
        user_te_labels.append(obj['user_data'][user]['y'])

In [5]:
te_data = np.concatenate(user_te_data, 0)
a = np.array(te_data)
te_data = (a-np.mean(a,1)[:,None])/np.std(a,1)[:,None]
te_labels = np.concatenate(user_te_labels)

te_len = len(te_labels)
if not os.path.exists('femnist_test_data_shuffler.pkl'):
    a = np.arange(te_len)
    np.random.shuffle(a)
    pickle.dump(a, open('femnist_test_data_shuffler.pkl', 'wb'))
else:
    a = pickle.load(open('femnist_test_data_shuffler.pkl', 'rb'))
te_data_tensor = torch.from_numpy(te_data[:(te_len//2)]).type(torch.FloatTensor)
te_label_tensor = torch.from_numpy(te_labels[:(te_len//2)]).type(torch.LongTensor)

val_data_tensor = torch.from_numpy(te_data[(te_len//2):]).type(torch.FloatTensor)
val_label_tensor = torch.from_numpy(te_labels[(te_len//2):]).type(torch.LongTensor)

In [20]:
each_worker_data=[]
each_worker_label=[]

each_worker_te_data=[]
each_worker_te_label=[]

if not os.path.exists('femnist_train_workers.pkl'):
    worker_idx = np.random.choice (3400, num_workers)
    pickle.dump(worker_idx, open('femnist_train_workers.pkl', 'wb'))
else:
    worker_idx = pickle.load(open('femnist_train_workers.pkl', 'rb'))
    
for i in worker_idx:
    data = np.array(user_tr_data[i])
    labels = np.array(user_tr_labels[i])
    
    te_data_ = np.array(user_te_data[i])
    te_labels_ = np.array(user_te_labels[i])
    
    data = (data-np.mean(data,1)[:,None])/np.std(data,1)[:,None]
    te_data_ = (te_data_-np.mean(te_data_,1)[:,None])/np.std(te_data_,1)[:,None]
    
    user_tr_data_tensor=torch.from_numpy(data).type(torch.FloatTensor)
    user_tr_label_tensor=torch.from_numpy(labels).type(torch.LongTensor)
    
    user_te_data_tensor=torch.from_numpy(te_data_).type(torch.FloatTensor)
    user_te_label_tensor=torch.from_numpy(te_labels_).type(torch.LongTensor)

    each_worker_data.append(user_tr_data_tensor)
    each_worker_label.append(user_tr_label_tensor)
    
    each_worker_te_data.append(user_te_data_tensor)
    each_worker_te_label.append(user_te_label_tensor)
    
    print('user %d tr len %d te len %d ' % (i, len(user_tr_data_tensor) ,len(user_te_data_tensor)))

global_te_data = torch.concat(each_worker_te_data)
global_te_label = torch.concat(each_worker_te_label)

user 1216 tr len 239 te len 27 
user 978 tr len 203 te len 23 
user 302 tr len 344 te len 39 
user 368 tr len 305 te len 34 
user 320 tr len 384 te len 43 
user 1384 tr len 253 te len 29 
user 138 tr len 333 te len 37 
user 2416 tr len 153 te len 17 
user 2686 tr len 165 te len 19 
user 1410 tr len 321 te len 36 
user 338 tr len 194 te len 22 
user 2180 tr len 141 te len 16 
user 2559 tr len 151 te len 17 
user 2645 tr len 163 te len 19 
user 2071 tr len 153 te len 18 
user 490 tr len 235 te len 27 
user 1130 tr len 69 te len 8 
user 1107 tr len 272 te len 31 
user 397 tr len 305 te len 34 
user 951 tr len 275 te len 31 
user 2299 tr len 148 te len 17 
user 2808 tr len 145 te len 17 
user 621 tr len 342 te len 38 
user 2378 tr len 153 te len 17 
user 2619 tr len 158 te len 18 
user 3 tr len 228 te len 26 
user 697 tr len 356 te len 40 
user 2406 tr len 153 te len 18 
user 2808 tr len 145 te len 17 
user 542 tr len 372 te len 42 
user 1437 tr len 242 te len 27 
user 716 tr len 337 te le

user 3220 tr len 159 te len 18 
user 3092 tr len 161 te len 18 
user 3304 tr len 155 te len 18 
user 2194 tr len 128 te len 15 
user 21 tr len 282 te len 32 
user 1229 tr len 255 te len 29 
user 2674 tr len 140 te len 16 
user 2688 tr len 161 te len 18 
user 2908 tr len 157 te len 18 
user 225 tr len 279 te len 31 
user 1240 tr len 211 te len 24 
user 3042 tr len 159 te len 18 
user 616 tr len 302 te len 34 
user 714 tr len 288 te len 33 
user 3074 tr len 160 te len 18 
user 174 tr len 375 te len 42 
user 2595 tr len 147 te len 17 
user 1255 tr len 259 te len 29 
user 2257 tr len 158 te len 18 
user 2463 tr len 159 te len 18 
user 331 tr len 320 te len 36 
user 599 tr len 325 te len 37 
user 898 tr len 165 te len 19 
user 1575 tr len 150 te len 17 
user 1066 tr len 226 te len 26 
user 81 tr len 299 te len 34 
user 2370 tr len 146 te len 17 
user 2063 tr len 104 te len 12 
user 1184 tr len 309 te len 35 
user 780 tr len 324 te len 36 
user 1357 tr len 186 te len 21 
user 3021 tr len 164

In [7]:
from torch.nn.utils import parameters_to_vector, vector_to_parameters

def train(train_data, labels, model, model_received, optimizer, batch_size=20):
    model.train()
    criterion = nn.CrossEntropyLoss()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    len_t = (len(train_data) // batch_size)
    if len(train_data)%batch_size:
        len_t += 1

    r=np.arange(len(train_data))
    np.random.shuffle(r)
    
    train_data = train_data[r]
    labels = labels[r]
    
    for ind in range(len_t):

        inputs = train_data[ind * batch_size:(ind + 1) * batch_size]
        targets = labels[ind * batch_size:(ind + 1) * batch_size]

        inputs, targets = inputs.cuda(), targets.cuda()

        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.item(), inputs.size(0))
        top1.update(prec1.item(), inputs.size(0))
        top5.update(prec5.item(), inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return (losses.avg, top1.avg)


def test(test_data, labels, model, criterion, use_cuda, debug_='MEDIUM', batch_size=64):
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    model.eval()
    len_t = (len(test_data) // batch_size)
    if len(test_data)%batch_size:
        len_t += 1

    with torch.no_grad():
        for ind in range(len_t):
            # measure data loading time
            inputs = test_data[ind * batch_size:(ind + 1) * batch_size]
            targets = labels[ind * batch_size:(ind + 1) * batch_size]

            inputs, targets = inputs.cuda(), targets.cuda()

            inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
            losses.update(loss.item(), inputs.size(0))
            top1.update(prec1.item(), inputs.size(0))
            top5.update(prec5.item(), inputs.size(0))

    return (losses.avg, top1.avg)

In [8]:
def lbfgs(S_k_list, Y_k_list, v):
    curr_S_k = torch.stack(S_k_list).T
    curr_Y_k = torch.stack(Y_k_list).T
    S_k_time_Y_k = np.dot(curr_S_k.T.cpu().numpy(), curr_Y_k.cpu().numpy())
    S_k_time_S_k = np.dot(curr_S_k.T.cpu().numpy(), curr_S_k.cpu().numpy())
    R_k = np.triu(S_k_time_Y_k)
    L_k = S_k_time_Y_k - R_k
    sigma_k = np.dot(Y_k_list[-1].unsqueeze(0).cpu().numpy(), S_k_list[-1].unsqueeze(0).T.cpu().numpy()) / (np.dot(S_k_list[-1].unsqueeze(0).cpu().numpy(), S_k_list[-1].unsqueeze(0).T.cpu().numpy()))
    D_k_diag = np.diag(S_k_time_Y_k)
    upper_mat = np.concatenate((sigma_k * S_k_time_S_k, L_k), axis=1)
    lower_mat = np.concatenate((L_k.T, -np.diag(D_k_diag)), axis=1)
    mat = np.concatenate((upper_mat, lower_mat), axis=0)
    mat_inv = np.linalg.inv(mat)

    approx_prod = sigma_k * v.cpu().numpy()
    approx_prod = approx_prod.T
    p_mat = np.concatenate((np.dot(curr_S_k.T.cpu().numpy(), sigma_k * v.unsqueeze(0).T.cpu().numpy()), np.dot(curr_Y_k.T.cpu().numpy(), v.unsqueeze(0).T.cpu().numpy())), axis=0)
    approx_prod -= np.dot(np.dot(np.concatenate((sigma_k * curr_S_k.cpu().numpy(), curr_Y_k.cpu().numpy()), axis=1), mat_inv), p_mat)

    return approx_prod

In [9]:
def full_trim(v, f):
    '''
    Full-knowledge Trim attack. w.l.o.g., we assume the first f worker devices are compromised.
    v: the list of squeezed gradients
    f: the number of compromised worker devices
    '''
    vi_shape = v[0].unsqueeze(0).T.shape
    v_tran = v.T
    
    maximum_dim = torch.max(v_tran, dim=1)
    maximum_dim = maximum_dim[0].reshape(vi_shape)
    minimum_dim = torch.min(v_tran, dim=1)
    minimum_dim = minimum_dim[0].reshape(vi_shape)
    direction = torch.sign(torch.sum(v_tran, dim=-1, keepdims=True))
    directed_dim = (direction > 0) * minimum_dim + (direction < 0) * maximum_dim

    for i in range(20):
        # apply attack to compromised worker devices with randomness
        ##random_12 = 1. + np.random.uniform(size=vi_shape)
        ##random_12 = torch.Tensor(random_12).float().cuda()
        random_12 = 2
        tmp = directed_dim * ((direction * directed_dim > 0) / random_12 + (direction * directed_dim < 0) * random_12)
        tmp = tmp.squeeze()
        v[i] = tmp
    return v

In [10]:
def tr_mean(all_updates, n_attackers):
    sorted_updates = torch.sort(all_updates, 0)[0]
    out = torch.mean(sorted_updates[n_attackers:-n_attackers], 0) if n_attackers else torch.mean(sorted_updates,0)
    return out

In [11]:
def simple_mean(old_gradients, user_grads, b=0, hvp=None):
    if hvp is not None:
        hvp = torch.from_numpy(hvp).to(device)
        pred_grad = copy.deepcopy(old_gradients)
        distance = []
        for i in range(len(old_gradients)):
            pred_grad[i] += hvp
        pred = np.zeros(100)
        pred[:b] = 1
        distance = torch.norm(pred_grad - user_grads, dim = 1).cpu().numpy()
        distance = distance / np.sum(distance)
    else:
        distance = None
    
    agg_grads = torch.mean(user_grads,dim=0)
    
    return agg_grads, distance

In [13]:
def trimmed_mean(old_gradients, user_grads, b=0, hvp=None):
    if hvp is not None:
        hvp = torch.from_numpy(hvp).to(device)
        pred_grad = copy.deepcopy(old_gradients)
        distance = []
        for i in range(len(old_gradients)):
            pred_grad[i] += hvp
        pred = np.zeros(100)
        pred[:b] = 1
        distance = torch.norm(pred_grad - user_grads, dim = 1).cpu().numpy()
        distance = distance / np.sum(distance)
    else:
        distance = None
    
    agg_grads = tr_mean(user_grads, 20)
    
    return agg_grads, distance

In [12]:
def median(old_gradients, user_grads, b=0, hvp=None):
    if hvp is not None:
        hvp = torch.from_numpy(hvp).to(device)
        pred_grad = copy.deepcopy(old_gradients)
        distance = []
        for i in range(len(old_gradients)):
            pred_grad[i] += hvp
        pred = np.zeros(100)
        pred[:b] = 1
        distance = torch.norm(pred_grad - user_grads, dim = 1).cpu().numpy()
        distance = distance / np.sum(distance)
    else:
        distance = None
    
    agg_grads = torch.median(user_grads, 0)[0]
    
    return agg_grads, distance

In [14]:
def detection(score, nobyz):
    estimator = KMeans(n_clusters=2)
    estimator.fit(score.reshape(-1, 1))
    label_pred = estimator.labels_
    if np.mean(score[label_pred==0])<np.mean(score[label_pred==1]):
        #0 is the label of malicious clients
        label_pred = 1 - label_pred
    real_label=np.ones(nworkers)
    real_label[:nobyz]=0
    acc=len(label_pred[label_pred==real_label])/nworkers
    recall=1-np.sum(label_pred[:nobyz])/nobyz
    fpr=1-np.sum(label_pred[nobyz:])/(nworkers-nobyz)
    fnr=np.sum(label_pred[:nobyz])/nobyz
    print("acc %0.4f; recall %0.4f; fpr %0.4f; fnr %0.4f;" % (acc, recall, fpr, fnr))
    print(silhouette_score(score.reshape(-1, 1), label_pred))

def detection1(score, nobyz):
    nrefs = 10
    ks = range(1, 8)
    gaps = np.zeros(len(ks))
    gapDiff = np.zeros(len(ks) - 1)
    sdk = np.zeros(len(ks))
    min = np.min(score)
    max = np.max(score)
    score = (score - min)/(max-min)
    for i, k in enumerate(ks):
        estimator = KMeans(n_clusters=k)
        estimator.fit(score.reshape(-1, 1))
        label_pred = estimator.labels_
        center = estimator.cluster_centers_
        Wk = np.sum([np.square(score[m]-center[label_pred[m]]) for m in range(len(score))])
        WkRef = np.zeros(nrefs)
        for j in range(nrefs):
            rand = np.random.uniform(0, 1, len(score))
            estimator = KMeans(n_clusters=k)
            estimator.fit(rand.reshape(-1, 1))
            label_pred = estimator.labels_
            center = estimator.cluster_centers_
            WkRef[j] = np.sum([np.square(rand[m]-center[label_pred[m]]) for m in range(len(rand))])
        gaps[i] = np.log(np.mean(WkRef)) - np.log(Wk)
        sdk[i] = np.sqrt((1.0 + nrefs) / nrefs) * np.std(np.log(WkRef))

        if i > 0:
            gapDiff[i - 1] = gaps[i - 1] - gaps[i] + sdk[i]
    #print(gapDiff)
    for i in range(len(gapDiff)):
        if gapDiff[i] >= 0:
            select_k = i+1
            break
    if select_k == 1:
        print('No attack detected!')
        return 0
    else:
        print('Attack Detected!')
        return 1

In [15]:
grad_list = []
old_grad_list = []
weight_record = []
grad_record = []
train_acc_list = []
distance1 = []
distance2 = []
auc_list = []

# Match bad FEMNIST baseline with 216 (72% of 300) clients (AGR = Mean)

# Mean

In [22]:
torch.cuda.empty_cache()
criterion = nn.CrossEntropyLoss()
net = cnn().to(device)
use_cuda = True
lr = 0.01
best_val_acc=0.0
for e in range(1000):
    # cnn_optimizer = SGD(net.parameters(), lr = lr*(0.995**e), weight_decay=1e-6)
    cnn_optimizer = SGD(net.parameters(), lr = lr)
    user_grads = []
    for i in range(nbyz, num_workers):
        net_ = copy.deepcopy(net)
        net_.zero_grad()
        output = net_(each_worker_data[i].reshape(-1, 1, 28, 28).cuda())
        loss = criterion(output, each_worker_label[i].cuda())
        loss.backward(retain_graph = True)
        param_grad=[]
        for param in net_.parameters():
            param_grad=param.grad.data.view(-1) if not len(param_grad) else torch.cat(
                (param_grad,param.grad.view(-1)))
        user_grads=param_grad[None, :] if len(user_grads)==0 else torch.cat((user_grads,param_grad[None,:]), 0)
        del net_

#     agg_grads = tr_mean(user_grads, 20)
#     agg_grads=torch.median(user_grads,dim=0)[0]
    agg_grads=torch.mean(user_grads,dim=0)

    del user_grads
    
    start_idx=0

    cnn_optimizer.zero_grad()

    model_grads=[]

    for i, param in enumerate(net.parameters()):
        param_=agg_grads[start_idx:start_idx+len(param.data.view(-1))].reshape(param.data.shape)
        start_idx=start_idx+len(param.data.view(-1))
        param_=param_.cuda()
        model_grads.append(param_)

    cnn_optimizer.step(model_grads)
    val_loss, val_acc = test(val_data_tensor.reshape(-1,1,28,28), val_label_tensor, net, criterion, use_cuda, batch_size=50)
    is_best = best_val_acc < val_acc
    best_val_acc = max(best_val_acc, val_acc)
    if is_best:
        _, best_te_acc = test(te_data_tensor.reshape(-1,1,28,28), te_label_tensor, net, criterion, use_cuda, batch_size=50)
    if e%10==0 or e==999:
        print('e %d | val_loss %.3f val acc %.3f | best val_acc %.2f test_acc %.2f' % (
                e, val_loss, val_acc, best_val_acc, best_te_acc))

e 0 | val_loss 4.125 val acc 5.591 | best val_acc 5.59 test_acc 3.93
e 10 | val_loss 3.833 val acc 13.329 | best val_acc 13.33 test_acc 6.93
e 20 | val_loss 3.551 val acc 13.545 | best val_acc 13.54 test_acc 6.67
e 30 | val_loss 3.423 val acc 20.538 | best val_acc 20.54 test_acc 9.71
e 40 | val_loss 3.366 val acc 25.234 | best val_acc 25.23 test_acc 11.90
e 50 | val_loss 3.316 val acc 28.164 | best val_acc 28.16 test_acc 13.96
e 60 | val_loss 3.261 val acc 30.555 | best val_acc 30.55 test_acc 15.50
e 70 | val_loss 3.198 val acc 33.015 | best val_acc 33.01 test_acc 16.85
e 80 | val_loss 3.128 val acc 35.435 | best val_acc 35.44 test_acc 18.22
e 90 | val_loss 3.051 val acc 37.821 | best val_acc 37.82 test_acc 19.49
e 100 | val_loss 2.970 val acc 40.184 | best val_acc 40.18 test_acc 20.64
e 110 | val_loss 2.885 val acc 42.299 | best val_acc 42.30 test_acc 21.92
e 120 | val_loss 2.798 val acc 44.262 | best val_acc 44.26 test_acc 23.34
e 130 | val_loss 2.712 val acc 45.847 | best val_acc 45

In [23]:
torch.cuda.empty_cache()
criterion = nn.CrossEntropyLoss()
net = cnn().to(device)
use_cuda = True
lr = 0.005
best_val_acc=0.0
for e in range(1000):
    # cnn_optimizer = SGD(net.parameters(), lr = lr*(0.995**e), weight_decay=1e-6)
    cnn_optimizer = SGD(net.parameters(), lr = lr)
    user_grads = []
    for i in range(nbyz, num_workers):
        net_ = copy.deepcopy(net)
        net_.zero_grad()
        output = net_(each_worker_data[i].reshape(-1, 1, 28, 28).cuda())
        loss = criterion(output, each_worker_label[i].cuda())
        loss.backward(retain_graph = True)
        param_grad=[]
        for param in net_.parameters():
            param_grad=param.grad.data.view(-1) if not len(param_grad) else torch.cat(
                (param_grad,param.grad.view(-1)))
        user_grads=param_grad[None, :] if len(user_grads)==0 else torch.cat((user_grads,param_grad[None,:]), 0)
        del net_

#     agg_grads = tr_mean(user_grads, 20)
#     agg_grads=torch.median(user_grads,dim=0)[0]
    agg_grads=torch.mean(user_grads,dim=0)

    del user_grads
    
    start_idx=0

    cnn_optimizer.zero_grad()

    model_grads=[]

    for i, param in enumerate(net.parameters()):
        param_=agg_grads[start_idx:start_idx+len(param.data.view(-1))].reshape(param.data.shape)
        start_idx=start_idx+len(param.data.view(-1))
        param_=param_.cuda()
        model_grads.append(param_)

    cnn_optimizer.step(model_grads)
    val_loss, val_acc = test(val_data_tensor.reshape(-1,1,28,28), val_label_tensor, net, criterion, use_cuda, batch_size=50)
    is_best = best_val_acc < val_acc
    best_val_acc = max(best_val_acc, val_acc)
    if is_best:
        _, best_te_acc = test(te_data_tensor.reshape(-1,1,28,28), te_label_tensor, net, criterion, use_cuda, batch_size=50)
    if e%10==0 or e==999:
        print('e %d | val_loss %.3f val acc %.3f | best val_acc %.2f test_acc %.2f' % (
                e, val_loss, val_acc, best_val_acc, best_te_acc))

e 0 | val_loss 4.120 val acc 0.750 | best val_acc 0.75 test_acc 1.66
e 10 | val_loss 4.026 val acc 4.933 | best val_acc 4.93 test_acc 2.73
e 20 | val_loss 3.922 val acc 6.301 | best val_acc 6.30 test_acc 3.13
e 30 | val_loss 3.797 val acc 7.314 | best val_acc 7.31 test_acc 3.40
e 40 | val_loss 3.661 val acc 11.167 | best val_acc 11.17 test_acc 4.78
e 50 | val_loss 3.546 val acc 16.945 | best val_acc 16.95 test_acc 8.18
e 60 | val_loss 3.472 val acc 22.276 | best val_acc 22.28 test_acc 10.90
e 70 | val_loss 3.430 val acc 26.301 | best val_acc 26.30 test_acc 12.94
e 80 | val_loss 3.402 val acc 30.607 | best val_acc 30.61 test_acc 15.28
e 90 | val_loss 3.378 val acc 32.900 | best val_acc 32.90 test_acc 16.57
e 100 | val_loss 3.356 val acc 34.065 | best val_acc 34.07 test_acc 17.19
e 110 | val_loss 3.333 val acc 34.920 | best val_acc 34.92 test_acc 17.73
e 120 | val_loss 3.309 val acc 35.710 | best val_acc 35.71 test_acc 18.06
e 130 | val_loss 3.283 val acc 36.330 | best val_acc 36.33 test

# Median

In [24]:
torch.cuda.empty_cache()
criterion = nn.CrossEntropyLoss()
net = cnn().to(device)
use_cuda = True
lr = 0.005
best_val_acc=0.0
for e in range(1000):
    # cnn_optimizer = SGD(net.parameters(), lr = lr*(0.995**e), weight_decay=1e-6)
    cnn_optimizer = SGD(net.parameters(), lr = lr)
    user_grads = []
    for i in range(nbyz, num_workers):
        net_ = copy.deepcopy(net)
        net_.zero_grad()
        output = net_(each_worker_data[i].reshape(-1, 1, 28, 28).cuda())
        loss = criterion(output, each_worker_label[i].cuda())
        loss.backward(retain_graph = True)
        param_grad=[]
        for param in net_.parameters():
            param_grad=param.grad.data.view(-1) if not len(param_grad) else torch.cat(
                (param_grad,param.grad.view(-1)))
        user_grads=param_grad[None, :] if len(user_grads)==0 else torch.cat((user_grads,param_grad[None,:]), 0)
        del net_

    agg_grads=torch.median(user_grads,dim=0)[0]

    del user_grads
    
    start_idx=0

    cnn_optimizer.zero_grad()

    model_grads=[]

    for i, param in enumerate(net.parameters()):
        param_=agg_grads[start_idx:start_idx+len(param.data.view(-1))].reshape(param.data.shape)
        start_idx=start_idx+len(param.data.view(-1))
        param_=param_.cuda()
        model_grads.append(param_)

    cnn_optimizer.step(model_grads)
    val_loss, val_acc = test(val_data_tensor.reshape(-1,1,28,28), val_label_tensor, net, criterion, use_cuda, batch_size=50)
    is_best = best_val_acc < val_acc
    best_val_acc = max(best_val_acc, val_acc)
    if is_best:
        _, best_te_acc = test(te_data_tensor.reshape(-1,1,28,28), te_label_tensor, net, criterion, use_cuda, batch_size=50)
    if e%10==0 or e==999:
        print('e %d | val_loss %.3f val acc %.3f | best val_acc %.2f test_acc %.2f' % (
                e, val_loss, val_acc, best_val_acc, best_te_acc))

e 0 | val_loss 4.107 val acc 4.356 | best val_acc 4.36 test_acc 2.10
e 10 | val_loss 3.972 val acc 6.684 | best val_acc 6.68 test_acc 3.39
e 20 | val_loss 3.817 val acc 6.701 | best val_acc 6.70 test_acc 3.39
e 30 | val_loss 3.648 val acc 6.724 | best val_acc 6.72 test_acc 3.39
e 40 | val_loss 3.518 val acc 11.614 | best val_acc 11.61 test_acc 4.97
e 50 | val_loss 3.449 val acc 21.768 | best val_acc 21.77 test_acc 10.73
e 60 | val_loss 3.411 val acc 26.066 | best val_acc 26.07 test_acc 13.06
e 70 | val_loss 3.381 val acc 28.884 | best val_acc 28.88 test_acc 14.54
e 80 | val_loss 3.352 val acc 31.037 | best val_acc 31.04 test_acc 15.44
e 90 | val_loss 3.322 val acc 32.682 | best val_acc 32.68 test_acc 16.22
e 100 | val_loss 3.291 val acc 34.078 | best val_acc 34.08 test_acc 16.91
e 110 | val_loss 3.258 val acc 35.448 | best val_acc 35.45 test_acc 17.54
e 120 | val_loss 3.223 val acc 36.688 | best val_acc 36.69 test_acc 18.11
e 130 | val_loss 3.186 val acc 37.686 | best val_acc 37.69 tes

In [25]:
torch.cuda.empty_cache()
criterion = nn.CrossEntropyLoss()
net = cnn().to(device)
use_cuda = True
lr = 0.01
best_val_acc=0.0
for e in range(1000):
    # cnn_optimizer = SGD(net.parameters(), lr = lr*(0.995**e), weight_decay=1e-6)
    cnn_optimizer = SGD(net.parameters(), lr = lr)
    user_grads = []
    for i in range(nbyz, num_workers):
        net_ = copy.deepcopy(net)
        net_.zero_grad()
        output = net_(each_worker_data[i].reshape(-1, 1, 28, 28).cuda())
        loss = criterion(output, each_worker_label[i].cuda())
        loss.backward(retain_graph = True)
        param_grad=[]
        for param in net_.parameters():
            param_grad=param.grad.data.view(-1) if not len(param_grad) else torch.cat(
                (param_grad,param.grad.view(-1)))
        user_grads=param_grad[None, :] if len(user_grads)==0 else torch.cat((user_grads,param_grad[None,:]), 0)
        del net_

    agg_grads=torch.median(user_grads,dim=0)[0]

    del user_grads
    
    start_idx=0

    cnn_optimizer.zero_grad()

    model_grads=[]

    for i, param in enumerate(net.parameters()):
        param_=agg_grads[start_idx:start_idx+len(param.data.view(-1))].reshape(param.data.shape)
        start_idx=start_idx+len(param.data.view(-1))
        param_=param_.cuda()
        model_grads.append(param_)

    cnn_optimizer.step(model_grads)
    val_loss, val_acc = test(val_data_tensor.reshape(-1,1,28,28), val_label_tensor, net, criterion, use_cuda, batch_size=50)
    is_best = best_val_acc < val_acc
    best_val_acc = max(best_val_acc, val_acc)
    if is_best:
        _, best_te_acc = test(te_data_tensor.reshape(-1,1,28,28), te_label_tensor, net, criterion, use_cuda, batch_size=50)
    if e%10==0 or e==999:
        print('e %d | val_loss %.3f val acc %.3f | best val_acc %.2f test_acc %.2f' % (
                e, val_loss, val_acc, best_val_acc, best_te_acc))

e 0 | val_loss 4.111 val acc 0.588 | best val_acc 0.59 test_acc 0.95
e 10 | val_loss 3.927 val acc 13.340 | best val_acc 13.40 test_acc 6.69
e 20 | val_loss 3.659 val acc 9.841 | best val_acc 13.40 test_acc 6.69
e 30 | val_loss 3.467 val acc 15.697 | best val_acc 15.70 test_acc 7.47
e 40 | val_loss 3.405 val acc 25.664 | best val_acc 25.66 test_acc 12.73
e 50 | val_loss 3.360 val acc 31.115 | best val_acc 31.11 test_acc 15.39
e 60 | val_loss 3.313 val acc 33.780 | best val_acc 33.78 test_acc 16.82
e 70 | val_loss 3.260 val acc 35.930 | best val_acc 35.93 test_acc 17.84
e 80 | val_loss 3.201 val acc 37.806 | best val_acc 37.81 test_acc 18.73
e 90 | val_loss 3.135 val acc 39.616 | best val_acc 39.62 test_acc 19.60
e 100 | val_loss 3.063 val acc 41.144 | best val_acc 41.14 test_acc 20.34
e 110 | val_loss 2.987 val acc 42.494 | best val_acc 42.49 test_acc 20.99
e 120 | val_loss 2.907 val acc 43.784 | best val_acc 43.78 test_acc 21.61
e 130 | val_loss 2.826 val acc 44.869 | best val_acc 44.

# Good FedAvg FEMNIST baseline with 216 (72% of 300) clients

In [26]:
from torch.nn.utils import parameters_to_vector, vector_to_parameters

def train(train_data, labels, model, optimizer, batch_size=20):
    model.train()
    criterion = nn.CrossEntropyLoss()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    len_t = (len(train_data) // batch_size)
    if len(train_data)%batch_size:
        len_t += 1

    r=np.arange(len(train_data))
    np.random.shuffle(r)
    
    train_data = train_data[r]
    labels = labels[r]
    
    for ind in range(len_t):

        inputs = train_data[ind * batch_size:(ind + 1) * batch_size]
        targets = labels[ind * batch_size:(ind + 1) * batch_size]

        inputs, targets = inputs.cuda(), targets.cuda()

        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
        losses.update(loss.item(), inputs.size(0))
        top1.update(prec1.item(), inputs.size(0))
        top5.update(prec5.item(), inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return (losses.avg, top1.avg)


def test(test_data, labels, model, criterion, use_cuda, debug_='MEDIUM', batch_size=64):
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    model.eval()
    len_t = (len(test_data) // batch_size)
    if len(test_data)%batch_size:
        len_t += 1

    with torch.no_grad():
        for ind in range(len_t):
            # measure data loading time
            inputs = test_data[ind * batch_size:(ind + 1) * batch_size]
            targets = labels[ind * batch_size:(ind + 1) * batch_size]

            inputs, targets = inputs.cuda(), targets.cuda()

            inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            prec1, prec5 = accuracy(outputs.data, targets.data, topk=(1, 5))
            losses.update(loss.item(), inputs.size(0))
            top1.update(prec1.item(), inputs.size(0))
            top5.update(prec5.item(), inputs.size(0))

    return (losses.avg, top1.avg)

# Mean

In [ ]:
torch.cuda.empty_cache()
nepochs=200
local_epochs = 3
batch_size = 10
num_workers = 300
local_lr = 0.1
global_lr = 1
nepochs = 50
nbyz = 28
byz_type = 'full_trim'
aggregation = 'trim'

criterion = nn.CrossEntropyLoss()
use_cuda = torch.cuda.is_available()

resume=False
round_nclients = num_workers
best_global_acc=0
epoch_num = 0
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)
fed_model = cnn().to(device)
net.apply(init_weights)

model_received = []
for i, (name, param) in enumerate(fed_model.state_dict().items()):
    model_received = param.view(-1).data.type(torch.cuda.FloatTensor) if len(model_received) == 0 else torch.cat((model_received, param.view(-1).data.type(torch.cuda.FloatTensor)))

while epoch_num <= nepochs:
    torch.cuda.empty_cache()
    round_clients = np.arange(nbyz, num_workers)
    round_benign = round_clients
    user_updates=[]
    benign_norm = 0
    
    for i in round_benign:
        model = copy.deepcopy(fed_model)
        optimizer = optim.SGD(model.parameters(), lr = local_lr*(0.999**epoch_num))
        # optimizer = optim.SGD(model.parameters(), lr = lr)

        for epoch in range(local_epochs):
            train_loss, train_acc = train(
                each_worker_data[i].reshape(-1, 1, 28, 28), torch.Tensor(each_worker_label[i]).long(),
                model, optimizer, batch_size)

        params = []
        for i, (name, param) in enumerate(model.state_dict().items()):
            params = param.view(-1).data.type(torch.cuda.FloatTensor) if len(params) == 0 else torch.cat(
                (params, param.view(-1).data.type(torch.cuda.FloatTensor)))

        update =  (params - model_received)
        benign_norm += torch.norm(update)/len(round_benign)
        user_updates = update[None,:] if len(user_updates) == 0 else torch.cat((user_updates, update[None,:]), 0)

    agg_update = torch.mean(user_updates, 0)

    del user_updates

    model_received = model_received + global_lr * agg_update
    fed_model = cnn().to(device)
    net.apply(init_weights)
    
    start_idx=0
    state_dict = {}
    previous_name = 'none'
    for i, (name, param) in enumerate(fed_model.state_dict().items()):
        start_idx = 0 if i == 0 else start_idx + len(fed_model.state_dict()[previous_name].data.view(-1))
        start_end = start_idx + len(fed_model.state_dict()[name].data.view(-1))
        params = model_received[start_idx:start_end].reshape(fed_model.state_dict()[name].data.shape)
        state_dict[name] = params
        previous_name = name

    fed_model.load_state_dict(state_dict)

    val_loss, val_acc = test(val_data_tensor.reshape(-1,1,28,28), val_label_tensor, fed_model, criterion, use_cuda, batch_size=50)
    is_best = best_val_acc < val_acc
    best_val_acc = max(best_val_acc, val_acc)
    
    if is_best:
        _, best_te_acc = test(te_data_tensor.reshape(-1,1,28,28), te_label_tensor, fed_model, criterion, use_cuda, batch_size=50)
    
    if epoch_num%10==0 or epoch_num==200:
        print('e %d | val_loss %.3f val acc %.3f | best val_acc %.2f test_acc %.2f' % (
                e, val_loss, val_acc, best_val_acc, best_te_acc))

    epoch_num+=1

# Setup FEMNIST baseline for mean and median for the given model

## This baseline is for FedSGD with full-batch gradients

# For mean

In [ ]:
torch.cuda.empty_cache()
criterion = nn.CrossEntropyLoss()
net = cnn().to(device)
use_cuda = True
lr = 0.25
best_val_acc=0.0
for e in range(100):
    cnn_optimizer = SGD(net.parameters(), lr = lr*(0.995**e), weight_decay=1e-6)
    user_grads = []
    for i in range(num_workers):
        net_ = copy.deepcopy(net)
        net_.zero_grad()
        output = net_(each_worker_data[i].reshape(-1, 1, 28, 28).cuda())
        loss = criterion(output, each_worker_label[i].cuda())
        loss.backward(retain_graph = True)
        param_grad=[]
        for param in net_.parameters():
            param_grad=param.grad.data.view(-1) if not len(param_grad) else torch.cat(
                (param_grad,param.grad.view(-1)))
        user_grads=param_grad[None, :] if len(user_grads)==0 else torch.cat((user_grads,param_grad[None,:]), 0)
        del net_

#     agg_grads = tr_mean(user_grads, 20)
#     agg_grads=torch.median(user_grads,dim=0)[0]
    agg_grads=torch.mean(user_grads,dim=0)

    del user_grads
    
    start_idx=0

    cnn_optimizer.zero_grad()

    model_grads=[]

    for i, param in enumerate(net.parameters()):
        param_=agg_grads[start_idx:start_idx+len(param.data.view(-1))].reshape(param.data.shape)
        start_idx=start_idx+len(param.data.view(-1))
        param_=param_.cuda()
        model_grads.append(param_)

    cnn_optimizer.step(model_grads)
    val_loss, val_acc = test(val_data_tensor.reshape(-1,1,28,28), val_label_tensor, net, criterion, use_cuda, batch_size=50)
    is_best = best_val_acc < val_acc
    best_val_acc = max(best_val_acc, val_acc)
    if is_best:
        _, best_te_acc = test(te_data_tensor.reshape(-1,1,28,28), te_label_tensor, net, criterion, use_cuda, batch_size=50)
    print('e %d | val_loss %.3f val acc %.3f | best val_acc %.2f test_acc %.2f' % (
        e, val_loss, val_acc, best_val_acc, best_te_acc))

# For median

In [ ]:
torch.cuda.empty_cache()
criterion = nn.CrossEntropyLoss()
net = cnn().to(device)
use_cuda = True
lr = 0.2
best_val_acc=0.0
for e in range(100):
    cnn_optimizer = SGD(net.parameters(), lr = lr*(0.995**e), weight_decay=1e-6)
    user_grads = []
    for i in range(num_workers):
        net_ = copy.deepcopy(net)
        net_.zero_grad()
        output = net_(each_worker_data[i].reshape(-1, 1, 28, 28).cuda())
        loss = criterion(output, each_worker_label[i].cuda())
        loss.backward(retain_graph = True)
        param_grad=[]
        for param in net_.parameters():
            param_grad=param.grad.data.view(-1) if not len(param_grad) else torch.cat(
                (param_grad,param.grad.view(-1)))
        user_grads=param_grad[None, :] if len(user_grads)==0 else torch.cat((user_grads,param_grad[None,:]), 0)
        del net_

#     agg_grads = tr_mean(user_grads, 20)
    agg_grads=torch.median(user_grads,dim=0)[0]
#     agg_grads=torch.mean(user_grads,dim=0)

    del user_grads
    
    start_idx=0

    cnn_optimizer.zero_grad()

    model_grads=[]

    for i, param in enumerate(net.parameters()):
        param_=agg_grads[start_idx:start_idx+len(param.data.view(-1))].reshape(param.data.shape)
        start_idx=start_idx+len(param.data.view(-1))
        param_=param_.cuda()
        model_grads.append(param_)

    cnn_optimizer.step(model_grads)
    val_loss, val_acc = test(val_data_tensor.reshape(-1,1,28,28), val_label_tensor, net, criterion, use_cuda, batch_size=50)
    is_best = best_val_acc < val_acc
    best_val_acc = max(best_val_acc, val_acc)
    if is_best:
        _, best_te_acc = test(te_data_tensor.reshape(-1,1,28,28), te_label_tensor, net, criterion, use_cuda, batch_size=50)
    print('e %d | val_loss %.3f val acc %.3f | best val_acc %.2f test_acc %.2f' % (
        e, val_loss, val_acc, best_val_acc, best_te_acc))

# FLDetector for FEMNIST + 300 clients + Cross-silo

In [ ]:
torch.cuda.empty_cache()
weight_record = []
grad_record = []
test_grads = []
malicious_scores = np.zeros((1, num_workers))
attack = False

criterion = nn.CrossEntropyLoss()
net = cnn().to(device)
lr = 0.25
best_val_acc=0.0

start_detection_epoch = 10
window_size = 10
assert (start_detection_epoch - window_size >= 0), 'start_detection_epoch %d should be more than window_size %d' % (start_detection_epoch, window_size)

for e in range(100):
    cnn_optimizer = SGD(net.parameters(), lr = lr*(0.995**e), weight_decay=1e-6)
    user_grads = []

    for i in range(num_workers):
        net_ = copy.deepcopy(net)
        net_.zero_grad()
        output = net_(each_worker_data[i].reshape(-1, 1, 28, 28).cuda())
        loss = criterion(output, each_worker_label[i].cuda())
        loss.backward(retain_graph = True)
        param_grad=[]
        for param in net_.parameters():
            param_grad=param.grad.data.view(-1) if not len(param_grad) else torch.cat((param_grad,param.grad.view(-1)))
        user_grads=param_grad[None, :] if len(user_grads)==0 else torch.cat((user_grads,param_grad[None,:]), 0)
        del net_
    tmp = []
    for param in net.parameters():
        tmp = param.data.view(-1) if not len(tmp) else torch.cat((tmp, param.data.view(-1)))
    weight = tmp    

    if (e > start_detection_epoch):
        hvp = lbfgs(weight_record, grad_record, weight - last_weight)
        hvp = np.squeeze(hvp)
    else:
        hvp = None

    if attack:
        if(e>0):
            user_grads = full_trim(user_grads, 20)
        elif e>10:
            for m in range(20):
                user_grads[m] = old_grad_list[m] + torch.from_numpy(hvp).to(device)

    agg_grads, distance = simple_mean(old_grad_list, user_grads, nbyz, hvp)
    
    if distance is not None and e > (start_detection_epoch - window_size):
        print('==>', e, malicious_scores.shape)
        malicious_scores = np.concatenate((malicious_scores, distance[None, :]), 0)

    if malicious_scores.shape[0] >= 11:
        print('performing detection at epoch %d' % e)
        if detection1(np.sum(malicious_scores[-10:], axis=0), 20):
            print('Stop at iteration:', e)
            detection(np.sum(malicious_scores[-10:], axis=0), 20)
            break

#     agg_grads = tr_mean(user_grads, 20)
#     agg_grads=torch.median(user_grads,dim=0)[0]
#     agg_grads=torch.mean(user_grads,dim=0)

    if e > (start_detection_epoch - window_size):
        weight_record.append(weight - last_weight)
        grad_record.append(agg_grads - last_grad)
    
    if (len(weight_record) > 10):
        del weight_record[0]
        del grad_record[0]
    
    last_weight = weight
    last_grad = agg_grads
    old_grad_list = user_grads
  
    del user_grads
    
    start_idx=0

    cnn_optimizer.zero_grad()

    model_grads=[]

    for i, param in enumerate(net.parameters()):
        param_=agg_grads[start_idx:start_idx+len(param.data.view(-1))].reshape(param.data.shape)
        start_idx=start_idx+len(param.data.view(-1))
        param_=param_.cuda()
        model_grads.append(param_)

    cnn_optimizer.step(model_grads)
    val_loss, val_acc = test(val_data_tensor.reshape(-1,1,28,28), val_label_tensor, net, criterion, use_cuda, batch_size=50)
    is_best = best_val_acc < val_acc
    best_val_acc = max(best_val_acc, val_acc)
    if is_best:
        _, best_te_acc = test(te_data_tensor.reshape(-1,1,28,28), te_label_tensor, net, criterion, use_cuda, batch_size=50)
    print('e %d | val_loss %.3f val acc %.3f | best val_acc %.2f test_acc %.2f' % (
        e, val_loss, val_acc, best_val_acc, best_te_acc))

# FLdetector + FEMNIST + naive trim attack

## Observation: FLdetector correctly identifies the malicious clients

In [ ]:
torch.cuda.empty_cache()
weight_record = []
grad_record = []
test_grads = []
malicious_scores = np.zeros((1, num_workers))
attack = True

criterion = nn.CrossEntropyLoss()
net = cnn().to(device)
lr = 0.25
best_val_acc=0.0

start_detection_epoch = 10
window_size = 10
assert (start_detection_epoch - window_size >= 0), 'start_detection_epoch %d should be more than window_size %d' % (start_detection_epoch, window_size)

for e in range(100):
    cnn_optimizer = SGD(net.parameters(), lr = lr*(0.995**e), weight_decay=1e-6)
    user_grads = []

    for i in range(num_workers):
        net_ = copy.deepcopy(net)
        net_.zero_grad()
        output = net_(each_worker_data[i].reshape(-1, 1, 28, 28).cuda())
        loss = criterion(output, each_worker_label[i].cuda())
        loss.backward(retain_graph = True)
        param_grad=[]
        for param in net_.parameters():
            param_grad=param.grad.data.view(-1) if not len(param_grad) else torch.cat((param_grad,param.grad.view(-1)))
        user_grads=param_grad[None, :] if len(user_grads)==0 else torch.cat((user_grads,param_grad[None,:]), 0)
        del net_
    tmp = []
    for param in net.parameters():
        tmp = param.data.view(-1) if not len(tmp) else torch.cat((tmp, param.data.view(-1)))
    weight = tmp    

    if (e > start_detection_epoch):
        hvp = lbfgs(weight_record, grad_record, weight - last_weight)
        hvp = np.squeeze(hvp)
    else:
        hvp = None

    if attack:
        if(e>0):
            user_grads = full_trim(user_grads, 20)

    agg_grads, distance = simple_mean(old_grad_list, user_grads, nbyz, hvp)
    
    if distance is not None and e > (start_detection_epoch - window_size):
        print('==>', e, malicious_scores.shape)
        malicious_scores = np.concatenate((malicious_scores, distance[None, :]), 0)

    if malicious_scores.shape[0] >= 11:
        print('performing detection at epoch %d' % e)
        if detection1(np.sum(malicious_scores[-10:], axis=0), 20):
            print('Stop at iteration:', e)
            detection(np.sum(malicious_scores[-10:], axis=0), 20)
            break

#     agg_grads = tr_mean(user_grads, 20)
#     agg_grads=torch.median(user_grads,dim=0)[0]
#     agg_grads=torch.mean(user_grads,dim=0)

    if e > (start_detection_epoch - window_size):
        weight_record.append(weight - last_weight)
        grad_record.append(agg_grads - last_grad)
    
    if (len(weight_record) > 10):
        del weight_record[0]
        del grad_record[0]
    
    last_weight = weight
    last_grad = agg_grads
    old_grad_list = user_grads
  
    del user_grads
    
    start_idx=0

    cnn_optimizer.zero_grad()

    model_grads=[]

    for i, param in enumerate(net.parameters()):
        param_=agg_grads[start_idx:start_idx+len(param.data.view(-1))].reshape(param.data.shape)
        start_idx=start_idx+len(param.data.view(-1))
        param_=param_.cuda()
        model_grads.append(param_)

    cnn_optimizer.step(model_grads)
    val_loss, val_acc = test(val_data_tensor.reshape(-1,1,28,28), val_label_tensor, net, criterion, use_cuda, batch_size=50)
    is_best = best_val_acc < val_acc
    best_val_acc = max(best_val_acc, val_acc)
    if is_best:
        _, best_te_acc = test(te_data_tensor.reshape(-1,1,28,28), te_label_tensor, net, criterion, use_cuda, batch_size=50)
    print('e %d | val_loss %.3f val acc %.3f | best val_acc %.2f test_acc %.2f' % (
        e, val_loss, val_acc, best_val_acc, best_te_acc))

# FLDetector + FEMNIST + hvp 

## Method: malicious clients compute updates that FLDetector would use for detection

In [ ]:
torch.cuda.empty_cache()
weight_record = []
grad_record = []
test_grads = []
malicious_scores = np.zeros((1, num_workers))
attack = True

criterion = nn.CrossEntropyLoss()
net = cnn().to(device)
lr = 0.25
best_val_acc=0.0
attack_type = 'none'
use_cuda = True
start_detection_epoch = 20
window_size = 10
assert (start_detection_epoch - window_size >= 0), 'start_detection_epoch %d should be more than window_size %d' % (start_detection_epoch, window_size)

for e in range(100):
    cnn_optimizer = SGD(net.parameters(), lr = lr*(0.995**e), weight_decay=1e-6)
    user_grads = []

    for i in range(num_workers):
        net_ = copy.deepcopy(net)
        net_.zero_grad()
        output = net_(each_worker_data[i].reshape(-1, 1, 28, 28).cuda())
        loss = criterion(output, each_worker_label[i].cuda())
        loss.backward(retain_graph = True)
        param_grad=[]
        for param in net_.parameters():
            param_grad=param.grad.data.view(-1) if not len(param_grad) else torch.cat((param_grad,param.grad.view(-1)))
        user_grads=param_grad[None, :] if len(user_grads)==0 else torch.cat((user_grads,param_grad[None,:]), 0)
        del net_
    tmp = []
    for param in net.parameters():
        tmp = param.data.view(-1) if not len(tmp) else torch.cat((tmp, param.data.view(-1)))
    weight = tmp    

    if (e > start_detection_epoch):
        hvp = lbfgs(weight_record, grad_record, weight - last_weight)
        hvp = np.squeeze(hvp)
    else:
        hvp = None

    if attack:
        if(e<start_detection_epoch):
            user_grads = full_trim(user_grads, nbyz)
        elif e>start_detection_epoch:
            for m in range(nbyz):
                if attack_type == 'LIE':
                    z = 0.2
                    noise_avg = torch.mean(user_grads[:nbyz], dim=0)
                    noise_std = torch.std(user_grads[:nbyz], dim=0)
                    noise = noise_avg + z * noise_std
                elif attack_type == 'NDSS21':
                    pass
                else:
                    noise = torch.zeros(hvp.shape)
                user_grads[m] = old_grad_list[m] + torch.from_numpy(hvp).to(device) + noise.to(device)

#     agg_grads, distance = simple_mean(old_grad_list, user_grads, nbyz, hvp)
    agg_grads, distance = trimmed_mean(old_grad_list, user_grads, nbyz, hvp)
    
    if distance is not None and e > (start_detection_epoch - window_size):
        print('==>', e, malicious_scores.shape)
        malicious_scores = np.concatenate((malicious_scores, distance[None, :]), 0)

    if malicious_scores.shape[0] >= 11:
        print('performing detection at epoch %d' % e)
        if detection1(np.sum(malicious_scores[-10:], axis=0), 20):
            print('Stop at iteration:', e)
            detection(np.sum(malicious_scores[-10:], axis=0), 20)
            break

#     agg_grads = tr_mean(user_grads, 20)
#     agg_grads=torch.median(user_grads,dim=0)[0]
#     agg_grads=torch.mean(user_grads,dim=0)

    if e > (start_detection_epoch - window_size):
        weight_record.append(weight - last_weight)
        grad_record.append(agg_grads - last_grad)
    
    if (len(weight_record) > 10):
        del weight_record[0]
        del grad_record[0]
    
    last_weight = weight
    last_grad = agg_grads
    old_grad_list = user_grads
  
    del user_grads
    
    start_idx=0

    cnn_optimizer.zero_grad()

    model_grads=[]

    for i, param in enumerate(net.parameters()):
        param_=agg_grads[start_idx:start_idx+len(param.data.view(-1))].reshape(param.data.shape)
        start_idx=start_idx+len(param.data.view(-1))
        param_=param_.cuda()
        model_grads.append(param_)

    cnn_optimizer.step(model_grads)
    val_loss, val_acc = test(val_data_tensor.reshape(-1,1,28,28), val_label_tensor, net, criterion, use_cuda, batch_size=50)
    is_best = best_val_acc < val_acc
    best_val_acc = max(best_val_acc, val_acc)
    if is_best:
        _, best_te_acc = test(te_data_tensor.reshape(-1,1,28,28), te_label_tensor, net, criterion, use_cuda, batch_size=50)
    print('e %d | val_loss %.3f val acc %.3f | best val_acc %.2f test_acc %.2f' % (
        e, val_loss, val_acc, best_val_acc, best_te_acc))

In [ ]:
np.sum(malicious_scores[-10:],0)